# 0. Import library

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

from statistics import mean

In [3]:
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
# from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
# from keras.preprocessing.text import Tokenizer
from keras.layers import Input
# from keras.layers.merge import Concatenate
# from keras.layers import Bidirectional

from keras.layers import concatenate

from keras.callbacks import EarlyStopping

# 1. Functions

In [4]:
#fuction to extract sequence features -- BLOSUM62
def BLOSUM62(sequences):
    blosum62 = {
        'A': [4,  -1, -2, -2, 0,  -1, -1, 0, -2,  -1, -1, -1, -1, -2, -1, 1,  0,  -3, -2, 0],  # A
        'R': [-1, 5,  0,  -2, -3, 1,  0,  -2, 0,  -3, -2, 2,  -1, -3, -2, -1, -1, -3, -2, -3], # R
        'N': [-2, 0,  6,  1,  -3, 0,  0,  0,  1,  -3, -3, 0,  -2, -3, -2, 1,  0,  -4, -2, -3], # N
        'D': [-2, -2, 1,  6,  -3, 0,  2,  -1, -1, -3, -4, -1, -3, -3, -1, 0,  -1, -4, -3, -3], # D
        'C': [0,  -3, -3, -3, 9,  -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1], # C
        'Q': [-1, 1,  0,  0,  -3, 5,  2,  -2, 0,  -3, -2, 1,  0,  -3, -1, 0,  -1, -2, -1, -2], # Q
        'E': [-1, 0,  0,  2,  -4, 2,  5,  -2, 0,  -3, -3, 1,  -2, -3, -1, 0,  -1, -3, -2, -2], # E
        'G': [0,  -2, 0,  -1, -3, -2, -2, 6,  -2, -4, -4, -2, -3, -3, -2, 0,  -2, -2, -3, -3], # G
        'H': [-2, 0,  1,  -1, -3, 0,  0,  -2, 8,  -3, -3, -1, -2, -1, -2, -1, -2, -2, 2,  -3], # H
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4,  2, -3, 1,  0, -3, -2, -1, -3, -1, 3],  # I
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2,  4,  -2, 2,  0, -3, -2, -1, -2, -1, 1],  # L
        'K': [-1, 2,  0,  -1, -3, 1,  1,  -2, -1, -3, -2, 5,  -1, -3, -1, 0,  -1, -3, -2, -2], # K
        'M': [-1, -1, -2, -3, -1, 0,  -2, -3, -2, 1,  2,  -1, 5,  0, -2, -1, -1, -1, -1, 1],  # M
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0,  0,  -3, 0,  6, -4, -2, -2, 1,  3,  -1], # F
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7,  -1, -1, -4, -3, -2], # P
        'S': [1,  -1, 1,  0,  -1, 0,  0,  0,  -1, -2, -2, 0,  -1, -2, -1, 4,  1,  -3, -2, -2], # S
        'T': [0,  -1, 0,  -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1,  5,  -2, -2, 0],  # T
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1, -4, -3, -2, 11, 2,  -3], # W
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2,  -1, -1, -2, -1, 3, -3, -2, -2, 2,  7,  -1], # Y
        'V': [0,  -3, -3, -3, -1, -2, -2, -3, -3, 3,  1,  -2, 1,  -1, -2, -2, 0,  -3, -1, 4],  # V
        '*': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # *
        'X': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # X
        'U': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # U
        '_': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # _
    }
    encodings = []
    for sequence in sequences:
        code=[]  
        for j in sequence:
            code = code + blosum62[j]
        encodings.append(np.array(code))       
    return encodings

#function to select negative data based on the composition of positive data
def Negative(df_neg_ST, df_neg_Y, positive):
    
    #the number of ST/Y site are consistent in positive and negative data
    if "Y" in positive["sequence"].str[7].value_counts().keys():
        num_Y = positive["sequence"].str[7].value_counts()["Y"]
    else:
        num_Y =0
    num_ST = len(positive)- num_Y

    negative_ST = df_neg_ST.sample(n=num_ST).reset_index(drop=True)
    negative_Y = df_neg_Y.sample(n=num_Y).reset_index(drop=True)
    negative = pd.concat([negative_ST, negative_Y], axis=0).reset_index(drop=True).sample(frac=1).reset_index(drop=True)

    negative["label"]=[0]*len(negative)

    #combine positive and negative data
    negative = negative.rename(columns={'protein neg': 'SubID', 'sequence neg': 'sequence'}) 
    
    return negative


#function to assign pathway and PPI encoding to corresponding items
def Embedding(feature, data, D):

    item =[]
    for i in range(0, len(data)):
        if data["SubID"][i] in feature.keys():
            item.append(feature[data["SubID"][i]])
        else:
            item.append(np.array([0]*D))
            
    return item
        

# 2. Read data

### 2.1 read understudied kinases and their highly similar kinases

In [5]:
#AGC group as an example, add kinases in other groups to train and test more models
dict_N = {
 'P22694': ['P17612'],
 'O75676': ['O75582', 'Q15349', 'P51812'],
 'Q96BR1': ['O00141'], 
 'Q9Y243': ['P31751', 'P31749'],
 'Q15208': ['Q9NRM7', 'O95835', 'Q9Y2H1'],
 'Q9UBS0': ['P23443', 'O75676', 'O75582', 'P51812', 'Q15349', 'Q15418']
}


### 2.2 read human K-S data and negative data

In [6]:
df = pd.read_csv("../0_data/Human K_S data.csv")

In [7]:
df_neg_Y = pd.read_csv("../0_data/negative Y sites.csv")
df_neg_ST = pd.read_csv("../0_data/negative ST sites.csv")

### 2.3 read pre-calculated pathway and ppi embeddings 

In [8]:
# a dictionary
pathway = np.load("../1_features/path_embedding.npy", allow_pickle=True)
pathway = pathway.flat[0]

# a dictionary
PPI = np.load("../1_features/sdne_embedding.npy", allow_pickle=True)
PPI = PPI.flat[0]

# 3. FCNN+LSTM model 

In [9]:
KIN =[]
K_accuracy2=[]
K_precision2=[]
K_recall2=[]
K_F1score2=[]
K_AUC2=[]

len_pos=[]


for k in dict_N.keys():
    
    #define empty lists to store performance metrics 
    Accuracy2=[]
    Precision2=[]
    Recall2 = []
    F1_score2=[]
    AUC2=[]

    #3.1.1 positive training data --- psites of highly similar kinases
    df1 = df[df["KIN_ACC"].isin(dict_N[k])].reset_index(drop=True)
    positive = df1[["SUB_ACC","PEPTIDE"]].reset_index(drop=True)
    positive["label"]=[1]*len(positive)
    positive = positive.rename(columns={'SUB_ACC': 'SubID', 'PEPTIDE': 'sequence'}) 
    
    
    #3.1.2 validate data --- psites of understudied kinases
    df_val = df[df["KIN_ACC"] ==k].reset_index(drop=True)
    val = df_val[["SUB_ACC","PEPTIDE"]].reset_index(drop=True)
    val["label"]=[1]*len(val)
    val=val[val["PEPTIDE"].str.len() == 15].reset_index(drop=True)
    val_positive = val.rename(columns={'SUB_ACC': 'SubID', 'PEPTIDE': 'sequence'})
        

    # for each understudied kinase, the process is repeated 10 times to get average performance 
    for i in range(0, 10):
        
        #3.1.3 negative training data --- different every time
        negative = Negative(df_neg_ST, df_neg_Y, positive)
        
        val_negative = Negative(df_neg_ST, df_neg_Y, val_positive)
        
        
        #3.1.4 train test split
        df_input = pd.concat([positive, negative], axis=0).reset_index(drop=True)
        df_input=df_input[df_input["sequence"].str.len() == 15].reset_index(drop=True)
        train = df_input.sample(frac=1).reset_index(drop=True)

        
        df_input_val = pd.concat([val_positive, val_negative], axis=0).reset_index(drop=True)
        validate = df_input_val.sample(frac=1).reset_index(drop=True)

        
        #3.2 ECODING
        #3.2.1 sequence
        X_seq_train = np.array(BLOSUM62(train["sequence"]))
        X_seq_validate = np.array(BLOSUM62(validate["sequence"]))
        #-----------------------------------------------
        
        #3.2.2 ppi
        X_ppi_train = np.array(Embedding(PPI, train, 128))
        X_ppi_validate = np.array(Embedding(PPI, validate, 128))

        #3.2.3 pathway
        X_path_train = np.array(Embedding(pathway, train, 347))
        X_path_validate = np.array(Embedding(pathway, validate, 347))
        #-----------------------------------------------

        Y_train = np.array(train["label"])
        Y_validate = np.array(validate["label"])
        #------------------------------------------------

        #=======================
        #3.1 Deep model
        input_pathway = Input(shape=(347,))
        input_ppi = Input(shape=(128,))

        #pathways
        model_pathway = Sequential()
        model_pathway = Dense(32, activation="relu")(input_pathway)
        model_pathway = Dense(4, activation="relu")(model_pathway)
        model_pathway = Model(inputs=input_pathway, outputs=model_pathway)

        #ppi
        model_ppi = Sequential()
        model_ppi = Dense(32, activation="relu")(input_ppi)
        model_ppi = Dense(4, activation="relu")(model_ppi)
        model_ppi = Model(inputs=input_ppi, outputs=model_ppi)

        #sequence
        model_seq = Sequential()
        model_seq.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(1, 300)))
        model_seq.add(LSTM(100, activation='relu', return_sequences=True))
        model_seq.add(LSTM(50, activation='relu', return_sequences=True))
        model_seq.add(LSTM(25, activation='relu'))
#        model_seq.add(Dense(20, activation='relu'))
#        model_seq.add(Dense(10, activation='relu'))


        combined = concatenate([model_seq.output, model_pathway.output, model_ppi.output])
        # apply a FC layer and then a regression prediction on the
        # combined outputs
        z = Dense(2, activation="relu")(combined)
        z = Dense(1, activation="sigmoid")(z)

        # then output a single value
        model_combined = Model(inputs=[model_seq.input, model_pathway.input, model_ppi.input], outputs=z)


        model_combined.compile(optimizer='adam', loss='mse')
        print(model_combined.summary())

        # patient early stopping
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
        model_combined.fit([X_seq_train.reshape(len(X_seq_train), 1, 300), X_path_train, X_ppi_train], Y_train, epochs=300, validation_split=0.2, batch_size=128, verbose=0, callbacks=[es])   


        validate_output = model_combined.predict([X_seq_validate.reshape(len(X_seq_validate), 1, 300), X_path_validate, X_ppi_validate])

        pred = np.array(np.concatenate(np.where(validate_output > 0.5, 1, 0)).flat)   

        target_names = ['neg 0', 'pos 1']
        #print(classification_report(Y_validate, preds, target_names=target_names))
        report = classification_report(Y_validate, pred, target_names=target_names, output_dict=True)
        acc2 = round(report["accuracy"],3)
        pre2 = round(report["weighted avg"]["precision"],3)
        recall2 = round(report["weighted avg"]["recall"],3)
        f12 = round(report["weighted avg"]["f1-score"],3)
        auc2 = roc_auc_score(Y_validate, pred, average='weighted')

        Accuracy2.append(acc2)
        Precision2.append(pre2)
        Recall2.append(recall2)
        F1_score2.append(f12)
        AUC2.append(auc2)
        
        
        
        
    KIN.append(k)

    K_accuracy2.append(mean(Accuracy2))
    K_precision2.append(mean(Precision2))
    K_recall2.append(mean(Recall2))
    K_F1score2.append(mean(F1_score2))
    K_AUC2.append(mean(AUC2))

    len_pos.append(len(positive))

2022-09-15 14:42:13.725096: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 1, 300)]     0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 1, 200)       400800      ['lstm_input[0][0]']             
                                                                                                  
 lstm_1 (LSTM)                  (None, 1, 100)       120400      ['lstm[0][0]']                   
                                                                                                  
 input_1 (InputLayer)           [(None, 347)]        0           []                               
                                                                                            

 input_6 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 lstm_10 (LSTM)                 (None, 1, 50)        30200       ['lstm_9[0][0]']                 
                                                                                                  
 dense_12 (Dense)               (None, 32)           11136       ['input_5[0][0]']                
                                                                                                  
 dense_14 (Dense)               (None, 32)           4128        ['input_6[0][0]']                
                                                                                                  
 lstm_11 (LSTM)                 (None, 25)           7600        ['lstm_10[0][0]']                
                                                                                                  
 dense_13 

                                                                                                  
 dense_27 (Dense)               (None, 4)            132         ['dense_26[0][0]']               
                                                                                                  
 concatenate_4 (Concatenate)    (None, 33)           0           ['lstm_19[0][0]',                
                                                                  'dense_25[0][0]',               
                                                                  'dense_27[0][0]']               
                                                                                                  
 dense_28 (Dense)               (None, 2)            68          ['concatenate_4[0][0]']          
                                                                                                  
 dense_29 (Dense)               (None, 1)            3           ['dense_28[0][0]']               
          

                                                                                                  
 lstm_27 (LSTM)                 (None, 25)           7600        ['lstm_26[0][0]']                
                                                                                                  
 dense_37 (Dense)               (None, 4)            132         ['dense_36[0][0]']               
                                                                                                  
 dense_39 (Dense)               (None, 4)            132         ['dense_38[0][0]']               
                                                                                                  
 concatenate_6 (Concatenate)    (None, 33)           0           ['lstm_27[0][0]',                
                                                                  'dense_37[0][0]',               
                                                                  'dense_39[0][0]']               
          

...

 input_100 (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 lstm_198 (LSTM)                (None, 1, 50)        30200       ['lstm_197[0][0]']               
                                                                                                  
 dense_294 (Dense)              (None, 32)           11136       ['input_99[0][0]']               
                                                                                                  
 dense_296 (Dense)              (None, 32)           4128        ['input_100[0][0]']              
                                                                                                  
 lstm_199 (LSTM)                (None, 25)           7600        ['lstm_198[0][0]']               
                                                                                                  
 dense_295

                                                                                                  
 dense_309 (Dense)              (None, 4)            132         ['dense_308[0][0]']              
                                                                                                  
 concatenate_51 (Concatenate)   (None, 33)           0           ['lstm_207[0][0]',               
                                                                  'dense_307[0][0]',              
                                                                  'dense_309[0][0]']              
                                                                                                  
 dense_310 (Dense)              (None, 2)            68          ['concatenate_51[0][0]']         
                                                                                                  
 dense_311 (Dense)              (None, 1)            3           ['dense_310[0][0]']              
          

Total params: 574,599
Trainable params: 574,599
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 15: early stopping
Model: "model_164"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_216_input (InputLayer)    [(None, 1, 300)]     0           []                               
                                                                                                  
 lstm_216 (LSTM)                (None, 1, 200)       400800      ['lstm_216_input[0][0]']         
                                                                                                  
 lstm_217 (LSTM)                (None, 1, 100)       120400      ['lstm_216[0][0]']               
                                                                                       

                                                                                                  
 input_113 (InputLayer)         [(None, 347)]        0           []                               
                                                                                                  
 input_114 (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 lstm_226 (LSTM)                (None, 1, 50)        30200       ['lstm_225[0][0]']               
                                                                                                  
 dense_336 (Dense)              (None, 32)           11136       ['input_113[0][0]']              
                                                                                                  
 dense_338 (Dense)              (None, 32)           4128        ['input_114[0][0]']              
          

 lstm_235 (LSTM)                (None, 25)           7600        ['lstm_234[0][0]']               
                                                                                                  
 dense_349 (Dense)              (None, 4)            132         ['dense_348[0][0]']              
                                                                                                  
 dense_351 (Dense)              (None, 4)            132         ['dense_350[0][0]']              
                                                                                                  
 concatenate_58 (Concatenate)   (None, 33)           0           ['lstm_235[0][0]',               
                                                                  'dense_349[0][0]',              
                                                                  'dense_351[0][0]']              
                                                                                                  
 dense_352

# 4. Results

In [10]:
gene={}
for i in range(0, len(df)):
    gene[df["KIN_ACC"][i]]=df["KINASE"][i]

G={}
for i in range(0, len(df)):
    G[df["KIN_ACC"][i]]=df["Group"][i]
    
gene_name=[]
group_name=[]
num_of_test=[]
for k in dict_N.keys():
    gene_name.append(gene[k])
    group_name.append(G[k])
    num_of_test.append(len(df[df["KIN_ACC"]==k].drop_duplicates(subset=["PEPTIDE"])))
    

In [11]:
results=pd.DataFrame()

results["kinase"] = KIN
results["gene names"]=gene_name
results["group"] = group_name
results["test size"]=num_of_test

results["Acc deep"]=K_accuracy2
results["Pre deep"]=K_precision2
results["Recall deep"]=K_recall2
results["F1 score deep"]=K_F1score2
results["AUC deep"]=K_AUC2

results["No of postive sites"] = len_pos


In [12]:
results.sort_values(by=["test size"]) # old results ----lll

,kinase,gene names,group,test size,Acc deep,Pre deep,Recall deep,F1 score deep,AUC deep,No of postive sites
0,P22694,PRKACB,AGC,6,0.9417,0.9537,0.9417,0.9406,0.941667,1031
1,O75676,RPS6KA4,AGC,6,0.8350,0.8329,0.8350,0.8142,0.835000,142
2,Q96BR1,SGK3,AGC,6,0.8000,0.8041,0.8000,0.7784,0.800000,89
5,Q9UBS0,RPS6KB2,AGC,9,0.8888,0.9040,0.8888,0.8877,0.888889,330
4,Q15208,STK38,AGC,10,0.7100,0.6720,0.7100,0.6707,0.710000,56
3,Q9Y243,AKT3,AGC,11,0.9001,0.9068,0.9001,0.8993,0.900000,528
